# This notebook performs topic modeling on lyrics to that we can investigate questions including : The differences between East Coast and West Coast rap

In [2]:
# common Python imports
import sys
import os
import re
from collections import defaultdict
import pickle
import datetime

In [3]:
# enable plotting in our notebook
import seaborn
import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline

In [4]:
# these are used for NLP, Data Manipulation, etc
import gensim
import nltk
import numpy as np
import pandas as pd
import seaborn as sns

c:\anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Load some NLTK data before we get started

In [5]:
%time nltk.download('punkt')
%time nltk.download('stopwords')
%time nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Wall time: 1.45 s
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\slick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Wall time: 20 ms
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\slick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Wall time: 68.8 ms


True

# And load a part-of-speech tagging model that was already trained

In [6]:
# helper function to translate POS tags from treebank to wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

In [7]:
# set up our lemmatizer in case we enable it
lemma = nltk.wordnet.WordNetLemmatizer()

# and load a POS tagger
# let's also load and test a Brill Part of Speech tagger which was trained on the Penn Treebank:
BRILL_TAGGER_FILE_PATH = 'resources/treebank_brill_aubt.pickle'
brill_tagger = pickle.load(open(BRILL_TAGGER_FILE_PATH, 'rb'))
print(brill_tagger)

# now let's kick the tires on this tagger
test_tag_tokens = 'The cat walked onto an airplane'.split()
print(brill_tagger.tag(test_tag_tokens))

print([lemma.lemmatize(x[0], get_wordnet_pos(x[1])) for x in brill_tagger.tag(test_tag_tokens)])

[('The', 'DT'), ('cat', '-None-'), ('walked', 'VBD'), ('onto', 'IN'), ('an', 'DT'), ('airplane', 'NN')]
['The', 'cat', 'walk', 'onto', 'an', 'airplane']


In [8]:
test_sentence_2 = 'Get me on the court and I\'m trouble Last week fucked around and got a triple double'
print(brill_tagger.tag(test_sentence_2.split()))

[('Get', '-None-'), ('me', 'PRP'), ('on', 'IN'), ('the', 'DT'), ('court', 'NN'), ('and', 'CC'), ("I'm", '-None-'), ('trouble', 'NN'), ('Last', 'JJ'), ('week', 'NN'), ('fucked', 'VBD'), ('around', 'IN'), ('and', 'CC'), ('got', 'VBD'), ('a', 'DT'), ('triple', 'RB'), ('double', 'VB')]


# Load some stopwords -- words which are commonly filtered out since they are common or do not carry much meaning

In [9]:
# load a stopword set we want to use...
stoplist = nltk.corpus.stopwords.words('english')
stop_word_set = set(stoplist)

# we'll remove a few more from our dataset
if False:
    stop_word_set.add('get')
    stop_word_set.add('got')
    stop_word_set.add('nigga')
    stop_word_set.add('niggas')
    stop_word_set.add('bitch')
    stop_word_set.add('fuck')
    stop_word_set.add('ain\t')
    stop_word_set.add('aint')

print(stop_word_set)

{'other', 'myself', 'should', 'between', 'being', 'your', "shan't", "mightn't", 'her', 'didn', 'here', 'to', 'the', 'while', 'after', 'such', 're', 've', 'haven', 'who', 'hadn', 'had', 'some', 'y', 'below', 'hasn', 'can', 'be', 'don', 'where', 'yourselves', 'm', 'most', 'weren', "didn't", 'through', 'as', 'did', "wouldn't", 't', 'whom', 'if', 'doesn', "don't", 'was', 'mightn', 'how', 'ours', "you've", 'd', 'under', 'yours', 'won', 'each', 'about', "you'll", 'because', 'in', 'own', 'you', 'have', 'wouldn', 'him', 'down', 'ain', "hadn't", 'are', 's', "should've", 'during', 'more', 'we', 'that', "haven't", 'again', 'why', 'herself', 'very', "you're", 'they', 'their', 'does', 'not', 'our', 'needn', 'too', 'its', 'she', 'no', 'ourselves', 'but', 'against', 'before', 'there', 'been', 'he', 'from', 'so', "won't", 'those', 'what', "that'll", 'it', 'with', 'ma', 'above', 'of', 'few', "needn't", 'further', "couldn't", 'i', 'o', 'hers', "weren't", 'these', 'mustn', 'shouldn', 'and', 'themselves',

# Loading the dataset
## This dataset comes from the Kaggle website at this URL: https://www.kaggle.com/artimous/every-song-you-have-heard-almost

In [10]:
%%time

print('Loading dataframes from CSV.  This might take some time...')

# NOTE : Without setting the engine here, we might hit the exception : "C error: EOF inside string ..."

# This dataset is comprised of two separate files possibly for size and download limitations
# so we'll put them together in a moment...
lyrics_1_df = pd.read_csv('c:/datasets/lyrics/lyrics1.csv',
                       engine = 'python')
lyrics_2_df = pd.read_csv('c:/datasets/lyrics/lyrics2.csv',
                       engine = 'python')
# now we can put them together into a single frame
lyrics_df = pd.concat([lyrics_1_df, lyrics_2_df])

print('Length of Set #1 : {}'.format(len(lyrics_1_df)))
print('Length of Set #2 : {}'.format(len(lyrics_2_df)))
print('Length of Both Set combined : {}'.format(len(lyrics_df)))

Loading dataframes from CSV.  This might take some time...
Length of Set #1 : 250000
Length of Set #2 : 266174
Length of Both Set combined : 516174
Wall time: 36.8 s


In [11]:
#replace carriage returns with periods to see if we can split lyrics as if they are sentences
lyrics_df = lyrics_df.replace({'\n': ' . '}, regex=True)

In [12]:
lyrics_df.head(10)

,Band,Lyrics,Song
0,Elijah Blake,"No, no . I ain't ever trapped out the bando . ...",Everyday
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,Elijah Blake,She don't live on planet Earth no more . She f...,The Otherside
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low ...",Pinot
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds
5,Elijah Blake,I just want to ready your mind . 'Cause I'll s...,Uno
6,Elijah Harris,To believe . Or not to believe . That is the q...,Girlfriend (Main)
7,Elijah Levi,"No one here can love or understand me . Oh, wh...",Bye Bye Blackbird
8,Elijah Levi,"Lullaby of Birdland, that's what I . Always h...",Lullaby of Birdland
9,Elijah Levi,I hate to see that evening sun go down . I hat...,St. Louis Blues


# Before we start do do any text analysis, let's figure out the Hip-Hop artists we have

In [13]:
# NOTE : Could not find the following in this set : 
# EAST COAST : Nas
# WEST COAST : Warren G, Tha Dogg Pound
east_coast_artists = ['The Notorious B.I.G.', 'Diddy', 'Wu-Tang Clan', 'Craig Mack', 'Tim Dog']
west_coast_artists = ['N.W.A', 'Dr. Dre', '2Pac', 'Eazy-E', 'Ice Cube', 'Snoop Dogg', 'Nate Dogg', 'Daz Dillinger', ]

In [14]:
artist_check_df = lyrics_df[lyrics_df['Band'].str.contains("D.P.")].groupby(['Band']).size()
print(artist_check_df)

Band
3D Picnic                          1
Benito DiPaula                     1
DMP Big Band                       7
DePaul University Jazz Ensemble    2
Jacki DePiro                       3
Mario DePriest                     1
Nicky DePaola                      4
Sidney DeParis                     3
dtype: int64


In [15]:
artist_song_check_df = lyrics_df[lyrics_df['Band'] == 'Craig Mack']['Song']
print(artist_song_check_df)

84884           Flava in Ya Ear
84885         Funk Wit da Style
84886                  Get Down
84887             Judgement Day
84888                  Mainline
84889    Making Moves With Puff
84890    Project: Funk da World
84891                  Real Raw
84892            When God Comes
Name: Song, dtype: object


In [16]:
conditions = [
    (lyrics_df['Band'].isin(east_coast_artists)),
    (lyrics_df['Band'].isin(west_coast_artists))]
choices = ['East', 'West']
lyrics_df['RapCoast'] = np.select(conditions, choices, default='')

In [17]:
rap_df = lyrics_df[lyrics_df['RapCoast'].str.len() > 0]

In [18]:
print(rap_df.groupby(['Band', 'RapCoast']).size())

Band                  RapCoast
2Pac                  West        252
Craig Mack            East          9
Daz Dillinger         West         22
Diddy                 East        104
Dr. Dre               West         80
Eazy-E                West         24
Ice Cube              West        178
N.W.A                 West         22
Nate Dogg             West         40
Snoop Dogg            West        344
The Notorious B.I.G.  East         90
Tim Dog               East          7
Wu-Tang Clan          East        125
dtype: int64


In [19]:
print(rap_df.groupby(['RapCoast']).size())

RapCoast
East    335
West    962
dtype: int64


In [20]:
# let's write this to a file
rap_artist_filename = 'rap_artists.csv'
rap_df.to_csv(rap_artist_filename)
print('Wrote Rap artist file to CSV : {}'.format(rap_artist_filename))

Wrote Rap artist file to CSV : rap_artists.csv


In [21]:
text_list = rap_df['Lyrics'].tolist()
lyrics_index_list = rap_df.index.values

# Now before we start working with the text, let's handle some of it's formatted.  Since many of the lyrics are back and forth between artists and sometimes parts of the songs are marked up, let's handle this by removing them so that they do not dominate our vocabulary.  Otherwise, we see the words 'snoop' and 'dogg' all over the topic model because he is featured in so many songs even if he is not the artist.  We'll do this with regular expressions:
* Replace [ARTIST NAME] with blanks
* Replace [chorus] with blanks
* etc

In [22]:
test_clean_string_1 = '[Bigg Snoop Dogg] Who let them Eastside ridaz out? [Dogg] [ Dre ]'
test_clean_string_2 = 'I am the verse [chorus] I am the part that repeats'

print('About to clean strings...')

# match brackets with up to 30 characters in between
bracket_pattern = re.compile('\[.{1,30}\]')
print(bracket_pattern.sub(' . ', test_clean_string_1))
print(bracket_pattern.sub(' . ', test_clean_string_2))

About to clean strings...
 .  Who let them Eastside ridaz out?  . 
I am the verse  .  I am the part that repeats


In [23]:
# process all of sentences for this formatting
characters_removed_list = []
for i in range(len(text_list)):
    original_text = text_list[i]
    text_length_before = len(original_text)
    clean_format_text = bracket_pattern.sub(' . ', original_text)
    text_length_after = len(clean_format_text)
    characters_removed = text_length_before - text_length_after
    characters_removed_list.append(characters_removed)
    text_list[i] = clean_format_text
    
print('Text cleaned of formatting')
print('Average characters removed per document : {}'.format(np.mean(np.array(characters_removed_list))))

#print(text_list[10])

Text cleaned of formatting
Average characters removed per document : 14.808018504240556


In [24]:
%%time

tokenized_texts = []
token_count_list = []
unique_token_set = set()
for i, text in enumerate(text_list):
    if i % 1000 == 0:
        print('Tokenization process : [{0}/{1}]'.format(i, len(text_list)))
        
    # get the index into the original text
    index = text_list[i]
        
    # there are lots and lots of rows which have no lyrics at all, so let's skip them
    if not isinstance(text, str):
        #print('Skipping column type : {0} at index {1}'.format(type(text), index))  
        continue
        
    # this is a better way to tokenize, but for the interest of time, we will tokenize with
    # whitespace using python's split() function
    tokens = nltk.word_tokenize(text.lower())
    #tokens = text.lower().split()
    tokenized_texts.append(tokens)
    token_count_list.append(len(tokens))
    unique_token_set |= set(tokens)
    
print('Total size of tokenized list : {}'.format(len(tokenized_texts)))
print('Total size of unique tokens : {}'.format(len(unique_token_set)))
print('DONE reading, tokenizing and counting')

Tokenization process : [0/1297]
Tokenization process : [1000/1297]
Total size of tokenized list : 1297
Total size of unique tokens : 24022
DONE reading, tokenizing and counting
Wall time: 27.5 s


# Before we go much further, let's set up a dataframe to gather statistics on our vocabulary including term frequency, term rarity using Inverse Document Frequency (IDF) and others

In [25]:
def get_term_stats_df(tokenized_texts, stop_word_set, enable_phrase_stats = False):
    # TODO : Is this really the best way to calculate TF-IDF?  By document?  Shouldn't this be by vocabulary?
    print('Building Dictionary...')
    dictionary = gensim.corpora.Dictionary(tokenized_texts)
    
    print('Setting up Bag-of-Words for [{0}] documents...'.format(len(tokenized_texts)))
    unfiltered_corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

    # now before we start using this, let's look at some term weighting with TF-IDF to see if there are some terms we can easily ignore
    # (e.g. patient, patient, cell, cells, etc) which occur very frequently across nearly all documents

    tfidf = gensim.models.TfidfModel(unfiltered_corpus, id2word=dictionary)
    
    print('TF-IDF model built and now setting up dataframe...')
    global_idf_map = {}
    term_dicts = []
    for id in tfidf.id2word.keys():
        word = tfidf.id2word[id]
        global_tf = tfidf.dfs[id]
        global_idf = tfidf.idfs[id]
        global_tfidf = global_tf * float(global_idf)
        global_idf_map[id] = global_tfidf
        
        stopword = word in stop_word_set
        
        any_alpha = any(c.isalpha() for c in word)
        all_alpha = word.isalpha()
        alpha_num = word.isalnum()
        phrase = '_' in word
        
        phrase_stopword_edge = False
        phrase_noalpha_edge = False
        phrase_disallowed_start = False
        if enable_phrase_stats and phrase:
            phrase_stopword_edge = phrase_has_stopword_edge(word, stop_word_set)
            phrase_noalpha_edge = phrase_has_no_alpha_edge(word)
            phrase_disallowed_start =  phrase_has_disallowed_start(word)

        term_dict = {'@Token' : word, 'Global TF' : global_tf, 
                     'Global TF-IDF' : global_tfidf, 'Global IDF' : global_idf,
                    'Stopword' : stopword, 'Phrase' : phrase,
                    'Any Alphabetic' : any_alpha, 'All Alphabetic' : all_alpha, 'All Alphanumeric' : alpha_num,
                    #'Phrase Stopword Edge' : phrase_stopword_edge, 'Phrase No Alpha Edge' : phrase_noalpha_edge,
                    #'Phrase Disallowed Start' : phrase_disallowed_start
                    }
        term_dicts.append(term_dict)

    term_stats_df = pd.DataFrame(term_dicts)
    term_stats_df = term_stats_df.sort_values('Global TF', ascending = False)
    return term_stats_df

In [26]:
%%time

print('Prepping stats for completely unfiltered vocabulary...')
unfiltered_term_stats_df = get_term_stats_df(tokenized_texts, stop_word_set)

display(unfiltered_term_stats_df.head(5))

Prepping stats for completely unfiltered vocabulary...
Building Dictionary...
Setting up Bag-of-Words for [1297] documents...
TF-IDF model built and now setting up dataframe...


,@Token,All Alphabetic,All Alphanumeric,Any Alphabetic,Global IDF,Global TF,Global TF-IDF,Phrase,Stopword
6475,.,False,False,False,0.001113,1296,1.442139,False,False
21812,the,True,True,True,0.011166,1287,14.371190,False,True
7697,i,True,True,True,0.012288,1286,15.802158,False,True
12701,to,True,True,True,0.014533,1284,18.660728,False,True
20599,and,True,True,True,0.022420,1277,28.630282,False,True


Wall time: 3.22 s


In [27]:
unfiltered_term_stats_filename = 'unfiltered_term_stats_{0}.csv'.format(datetime.datetime.now().strftime("%m_%d"))
print('Writing Term Stats CSV to : {}'.format(unfiltered_term_stats_filename))

unfiltered_term_stats_df.to_csv(unfiltered_term_stats_filename)

print('DONE Writing Term Stats CSV to : {}'.format(unfiltered_term_stats_filename))

Writing Term Stats CSV to : unfiltered_term_stats_06_30.csv
DONE Writing Term Stats CSV to : unfiltered_term_stats_06_30.csv


In [28]:
# Let's plot the distribution of terms as well
#global_idf_series = pd.Series(unfiltered_term_stats_df['Global IDF'], name='Global IDF')
        
# let's look at the distribution of TF-IDF values
#seaborn.distplot(global_idf_series);

# Now we'll process and clean the texts before we train a topic model...

In [29]:
class Token:
    def __init__(self, token, pos):
        self.token = token
        self.pos = pos
        
    def __repr__(self):
        return '{0}/{1}'.format(self.token, self.pos)

In [30]:
CULL_ARTIST_NAMES = True

In [31]:
MIN_TERM_FREQUENCY = 3
MIN_TOKEN_LENGTH = 3
CULL_STOP_WORDS = True
CULL_BY_MIN_FREQUENCY = True
CULL_NON_ALPHA = False
CULL_TOKENS_WITH_NO_ALPHA = True
CULL_SHORT_TOKENS = True
CULL_PHRASES_WITH_STOPWORD_EDGE = False
CULL_PHRASE_NO_ALPHA_EDGE = False
CULL_PHRASE_DISALLOWED_START = False
CULL_NON_NOUNS = True
FILTER_DICTIONARY_BY_GLOBAL_IDF = True
MIN_GLOBAL_IDF_VALUE = 3.0

In [32]:
# but for now we'll use the original text verbatim
filtered_tokenized_texts = tokenized_texts

print(filtered_tokenized_texts[0][:50])

['.', '.', 'live', 'in', 'this', 'motherfucker', 'thug', 'pound', 'bitch', '.', 'kurupt', 'daz', 'makaveli', 'idi', 'kastro', 'hussein', '.', 'you', 'know', 'bitch', 'the', 'whole', 'muthafuckin', "'", 'clique', '.', '.', 'we', 'gon', "'", 'hit', 'yo', "'", 'ass', 'up', '.', '.', 'you', 'know', 'where', 'we', 'comin', "'", 'from', '.', 'death', 'row', 'bitch', 'ay', 'yo']


In [46]:
if CULL_ARTIST_NAMES:
    all_artist_name_set = set(rap_df['Band'].tolist())
    print('Adding [{}] artist names to STOP WORD LIST...'.format(len(all_artist_name_set)))
    for artist_name in all_artist_name_set:
        artist_tokens = artist_name.split()
        for artist_token in artist_tokens:
            stop_word_set.add(artist_token.lower())
            
            
    # let's also add some other variants of artist names that may be different from the names in this set
    stop_word_set.add('pac')
    stop_word_set.add('2-pac')
    stop_word_set.add('2pac')
    stop_word_set.add('biggie')
    stop_word_set.add('smalls')
    stop_word_set.add('smallz')
    stop_word_set.add('dogg')
    stop_word_set.add('doggy')
    
    print('Total updates stopword set size : {}'.format(len(stop_word_set)))

Adding [13] artist names to STOP WORD LIST...
Total updates stopword set size : 206


In [37]:
# convert everything into this class even if we do not do POS tagging
if CULL_NON_NOUNS:
    print('Preparing Token and POS information for later culling...')
    for i in range(len(filtered_tokenized_texts)):
        document_tokens = filtered_tokenized_texts[i]
        token_objects = [Token(pos[0], pos[1]) for pos in brill_tagger.tag(document_tokens)]
        filtered_tokenized_texts[i] = token_objects
        
        #print(document_tokens)
        #break
    
    print('DONE with part of speech tagging')
    
else:
    print('Preparing Token information even without culling later')
    for i in range(len(filtered_tokenized_texts)):
        document_tokens = filtered_tokenized_texts[i]
        token_objects = [Token(token, '') for token in document_tokens if len(token) > 0]
        # store this back in 
        filtered_tokenized_texts[i] = token_objects
    
#print(filtered_tokenized_texts[:10])
print(filtered_tokenized_texts[0][:50])

Preparing Token and POS information for later culling...
DONE with part of speech tagging
[./., ./., live/VB, in/IN, this/DT, motherfucker/NN, thug/-None-, pound/NN, bitch/NN, ./., kurupt/JJ, daz/-None-, makaveli/-None-, idi/-None-, kastro/-None-, hussein/NNP, ./., you/PRP, know/VBP, bitch/NN, the/DT, whole/JJ, muthafuckin/NNP, '/POS, clique/NNP, ./., ./., we/PRP, gon/JJ, '/POS, hit/VBN, yo/-None-, '/POS, ass/NN, up/IN, ./., ./., you/PRP, know/VBP, where/WRB, we/PRP, comin/NNP, '/POS, from/IN, ./., death/NN, row/NN, bitch/NN, ay/-None-, yo/-None-]


In [38]:
%%time

# prepare to remove words that appear only once
frequency = defaultdict(int)
for text in filtered_tokenized_texts:
    for token in text:
        frequency[token.token] += 1
        
if CULL_STOP_WORDS:
    print('Culling stopwords...')
    # then CULL by alpha
    filtered_tokenized_texts = [[token for token in text if token.token not in stop_word_set]
             for text in filtered_tokenized_texts]
    
if CULL_NON_NOUNS:
    # keep this for any noun or any phrase ('_') since phrases may not be properly labeled for part of speech
    filtered_tokenized_texts = [[token for token in text if ('_' in token.token or get_wordnet_pos(token.pos) == nltk.corpus.wordnet.NOUN)]
             for text in filtered_tokenized_texts]
    
if CULL_PHRASES_WITH_STOPWORD_EDGE:
    print('Culling phrases with a stopword on their EDGE')
    filtered_tokenized_texts = [[token for token in text if not phrase_has_stopword_edge(token.token, stop_word_set)]
             for text in filtered_tokenized_texts]
    
if CULL_PHRASE_NO_ALPHA_EDGE:
    print('Culling phrases with a NO ALPHA term on their EDGE')
    filtered_tokenized_texts = [[token for token in text if not phrase_has_no_alpha_edge(token.token)]
             for text in filtered_tokenized_texts]
    
if CULL_PHRASE_DISALLOWED_START:
    print('Culling phrases starting with a DISALLOWED term (e.g. "p", "n")')
    filtered_tokenized_texts = [[token for token in text if not phrase_has_disallowed_start(token.token)]
             for text in filtered_tokenized_texts]

if CULL_BY_MIN_FREQUENCY:
    print('Culling by min frequency...')
    # CULL by frequency
    filtered_tokenized_texts = [[token for token in text if frequency[token.token] > MIN_TERM_FREQUENCY]
             for text in filtered_tokenized_texts]
    
# NOTE that this culling only culls tokens which have NO alpha tokens at all
# which is very different that the processing below
# this will enable this:
# 'TP53' whereas the method below would cull it
if CULL_TOKENS_WITH_NO_ALPHA:
    filtered_tokenized_texts = [[token for token in text if any(c.isalpha() for c in token.token)]
             for text in filtered_tokenized_texts]

if CULL_NON_ALPHA:
    print('Culling all non-alpha tokens...')
    # then CULL by alpha
    filtered_tokenized_texts = [[token for token in text if token.token.isalpha()]
             for text in filtered_tokenized_texts]
    
if CULL_SHORT_TOKENS:
    filtered_tokenized_texts = [[token for token in text if len(token.token) >= MIN_TOKEN_LENGTH]
             for text in filtered_tokenized_texts]
    
# finally convert back from the object into simple token strings
filtered_tokenized_texts = [[token.token for token in text] for text in filtered_tokenized_texts]

DEMO_WORDS_TO_PRINT = 50

print('Demo document tokens:')
print(filtered_tokenized_texts[0][:DEMO_WORDS_TO_PRINT])

Culling stopwords...
Culling by min frequency...
Demo document tokens:
['motherfucker', 'thug', 'pound', 'bitch', 'makaveli', 'kastro', 'hussein', 'bitch', 'muthafuckin', 'clique', 'ass', 'comin', 'death', 'row', 'bitch', 'nigga', 'westside', 'nigga', 'ballers', 'steel', 'cowards', 'panic', 'button', 'thinkin', 'wad', 'bomb', 'vietnam', 'rhyme', 'spine', 'back', 'cracker', 'smack', 'grammar', 'murder', 'alabama', 'arm', 'hammer', 'across', 'niggas', 'like', 'cause', 'niggas', 'fit', 'show', 'hoe', 'proposal', 'bitch', 'mashin', 'ford', 'hell']
Wall time: 3.17 s


# Set up our initial dictionary

In [39]:
dictionary = gensim.corpora.Dictionary(filtered_tokenized_texts)
print('Total dictionary size : [{}]'.format(len(dictionary.keys())))

Total dictionary size : [5327]


# But before we go on, let's see if we should filter our vocabulary of very common terms (IDF)

In [40]:
def filter_dictionary_by_global_idf(corpus, tokenized_texts, tfidf_model, dictionary, min_global_idf_value, 
                                    print_low_value_words = True):
    low_value_words = set()
    low_value_ids = set()
    for id in tfidf.id2word.keys():
        word = tfidf.id2word[id]
        global_tf = tfidf.dfs[id]
        global_idf = tfidf.idfs[id]
        if global_idf < min_global_idf_value:
            low_value_words.add(word)
            low_value_ids.add(id)
            
    print('Total number of words removed : [{}]'.format(len(low_value_words)))

    if print_low_value_words:
        print('Here are all the words we will remove from our dictionary by global IDF')
        print(sorted(list(low_value_words)))
        
        low_value_word_percent = len(low_value_words) / float(len(dictionary))

        print('Total low value words : {0}'.format(len(low_value_words)))
        print('Total low value Total Vocab percent : {0}'.format(low_value_word_percent))
        
    # now we can filter
    dictionary.filter_tokens(bad_ids=low_value_ids)

In [41]:
# NOTE : This class was taken from an example here:
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb

class Corpus(gensim.corpora.textcorpus.TextCorpus):

    def get_texts(self):
        # let's also make sure that all empty documents (no tokens) are not included 
        # NOTE : The DIM model (model = 'fixed') breaks if there are any documents with no tokens
        # also, apparently the logic for checking this is any(not text), so let's try that
        #print('Calling a version of get_texts() that should not allow empty/None lists...')
        return [x for x in self.input]

    def __len__(self):
        return len(self.input)

In [42]:
corpus = Corpus(filtered_tokenized_texts)

# this is a corpus which has not yet been pruned so we can do meaningful TF-IDF calculations with it
# NOTE that we call "token.token" since it is of class token and we want the string for the actual token
unfiltered_corpus = [dictionary.doc2bow([token.token for token in tokens]) for tokens in tokenized_texts]

tfidf = gensim.models.TfidfModel(unfiltered_corpus, id2word=dictionary)

print('Corpus length : {}'.format(len(corpus)))

if FILTER_DICTIONARY_BY_GLOBAL_IDF:
    print('Total dictionary size BEFORE TF-IDF filtering : [{}]'.format(len(dictionary.keys())))
    
    filter_dictionary_by_global_idf(unfiltered_corpus, filtered_tokenized_texts, tfidf, dictionary, MIN_GLOBAL_IDF_VALUE)
    print(dictionary)
    print(corpus.dictionary)
    # re-point the new dictionary for the corpus
    corpus.dictionary = dictionary
    
    print('Total dictionary size AFTER TF-IDF filtering : [{}]'.format(len(corpus.dictionary.keys())))

Corpus length : 1297
Total dictionary size BEFORE TF-IDF filtering : [5327]
Total number of words removed : [155]
Here are all the words we will remove from our dictionary by global IDF
["'cause", "'em", "'ll", 'act', 'another', 'around', 'ass', 'baby', 'back', 'bang', 'bitch', 'bitches', 'black', 'block', 'blow', 'body', 'bout', 'boy', 'bust', 'call', 'cash', 'catch', 'cause', 'change', 'close', 'cold', 'comin', 'could', 'crazy', 'day', 'death', 'dick', 'dope', 'drop', 'every', 'everybody', 'everything', 'eyes', 'face', 'flow', 'fool', 'fuck', 'fuckin', 'game', 'gangsta', 'gettin', 'girl', 'god', 'guess', 'hand', 'hands', 'hate', 'head', 'heart', 'hell', 'hey', 'high', 'hit', 'hoe', 'home', 'homies', 'house', "i'ma", 'kick', 'life', 'like', 'livin', 'long', 'look', 'lookin', 'lot', 'low', 'mad', 'makin', 'mama', 'man', 'might', 'mind', 'mine', 'money', 'motherfucker', 'motherfuckers', 'motherfuckin', 'move', 'must', 'name', 'need', 'next', 'nigga', 'niggas', 'niggaz', 'night', 'nobody

In [43]:
%%time 

NUM_TOPICS = 15
NUM_WORKERS = 6
VANILLA_LDA_PASSES = 5

TRAIN_MULTICORE_MODEL = False

# train model
lda = None
if TRAIN_MULTICORE_MODEL:
    print('Training multicore model...')
    lda = gensim.models.LdaMulticore(corpus, 
                                     id2word = dictionary, 
                                     num_topics = NUM_TOPICS, 
                                     workers = NUM_WORKERS, 
                                     passes = VANILLA_LDA_PASSES)
else:
    print('Training single-core model...')
    lda = gensim.models.Lda(corpus, 
                                     id2word = dictionary, 
                                     num_topics = NUM_TOPICS)

Wall time: 1min 15s


In [44]:
lda.print_topics(-1)

[(0,
  '0.018*"n****" + 0.015*"days" + 0.012*"n****s" + 0.010*"f***" + 0.008*"part" + 0.008*"end" + 0.007*"vapors" + 0.007*"trip" + 0.007*"count" + 0.007*"gang"'),
 (1,
  '0.018*"thang" + 0.017*"ballin" + 0.009*"picture" + 0.007*"cha" + 0.006*"gold" + 0.006*"dollar" + 0.006*"pimp" + 0.005*"nuts" + 0.005*"collar" + 0.005*"wake"'),
 (2,
  '0.012*"sky" + 0.010*"tang" + 0.008*"soulja" + 0.007*"limit" + 0.007*"family" + 0.006*"batman" + 0.006*"mansion" + 0.005*"westside" + 0.005*"peace" + 0.005*"years"'),
 (3,
  '0.014*"city" + 0.012*"lord" + 0.011*"enemies" + 0.010*"cop" + 0.009*"cali" + 0.009*"bye" + 0.006*"california" + 0.005*"party" + 0.005*"outlaw" + 0.004*"park"'),
 (4,
  '0.012*"friends" + 0.011*"murder" + 0.010*"homie" + 0.007*"mourn" + 0.007*"doggz" + 0.006*"lady" + 0.005*"dup" + 0.005*"dumb" + 0.005*"till" + 0.004*"dollars"'),
 (5,
  '0.010*"pac" + 0.010*"cha" + 0.010*"till" + 0.009*"somebody" + 0.008*"doggy" + 0.007*"loot" + 0.007*"funk" + 0.006*"gim" + 0.006*"mainline" + 0.006*"